In [ ]:
%cd /gpfs/space/home/danylobo/bm-ai-pipelines/common/ocs/lightning_pipeline

In [ ]:
import cv2
import torch
import torchmetrics
import numpy as np 
from tqdm import tqdm
from inference_utils import predict_frame, get_model, draw_contours_from_mask, filter_axes, imshow
from pathlib import Path
from dataset.dataset import OneclickDataset

import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 150
plt.show()

%load_ext autoreload
%autoreload 2

In [ ]:
ndim = 2

version = 30
ckpt_dir = Path(f"lightning_logs/2D-segm/version_{version}/checkpoints")
ckpt_path = list(ckpt_dir.iterdir())[0]

device = torch.device('cuda:0')

In [ ]:
# val set:
img_dir = Path("/gpfs/space/projects/BetterMedicine/danylo/lung/training/raw/2D-001_LIDC_thin_CropPad_consensus1/Ts/images")
# test set thick=2.5
img_dir = Path("/gpfs/space/projects/BetterMedicine/danylo/lung/training/raw/2D-001_LIDC_thick=2_5_CropPad_consensus1-Tr/images")
# test 201 redbrick
img_dir = Path("/gpfs/space/projects/BetterMedicine/danylo/lung/training/raw/2D_201_redbrick/images")


dst_dir = Path("predictions/test_set_201_119")
dst_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
model = get_model(ckpt_path)

In [ ]:
val_files = img_dir.rglob("*npy")
val_files = [str(filepath) for filepath in val_files]
val_files = filter_axes(val_files)

In [ ]:
add_input = {'point_prompt': 2,
             'prob_random_pair': 0.,
             'prob_jitter': 0.}

dataset = OneclickDataset(files=val_files,
                          ndim=ndim,
                          mode="test",
                          add_input=add_input)

val_dice = torchmetrics.Dice()

for img_tensor, mask_tensor in tqdm(dataset):
    img_tensor = img_tensor.unsqueeze(0).to(device=device)

    pred = predict_frame(model=model, img_tensor=img_tensor, ndim=ndim)
    val_dice.update(torch.tensor(pred), mask_tensor.squeeze().to(torch.int8))
    
    img = img_tensor.squeeze().detach().cpu().numpy()
    gt = mask_tensor.squeeze().detach().cpu().numpy()

    # # Uncomment to visualize

    # img = img[0] if add_input['point_prompt'] > 0 else img
    # img = 255 * cv2.cvtColor(0.226 * img + 0.449, cv2.COLOR_GRAY2RGB)
    # img = img.astype(np.uint8)

    # img_pred = draw_contours_from_mask(img, pred,  [255, 20, 147])
    # img_gt   = draw_contours_from_mask(img, gt, [0, 255, 102])
    
    # # separators
    # img_pred[:, 0] = 255
    # img_gt[:, 0] = 255

    # img_stacked = np.hstack([img, img_pred, img_gt])
    # imshow(img_stacked)

val_dice_mean = val_dice.compute()
val_dice.reset()

print(val_dice_mean)